In [42]:
import pandas as pd
import random
import numpy as np
import glob
import os
from unrar import rarfile
import py7zr
import pickle
import datetime
import time
pd.set_option("max_columns", 200)


year = "2020"
startDate = '20200103'
endDate = '20200103'
readPath = 'L:\\DB\\' + year + '\\SZ\\***'
dataPathLs = np.array(glob.glob(readPath))
dateLs = np.array([os.path.basename(i) for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]

for data in dataPathLs:
    
    startTm = datetime.datetime.now()
    
    readPath = data + '\\order\\***'
    dataPathLs = np.array(glob.glob(readPath))
    dateLs = np.array([int(os.path.basename(i).split('.')[0]) for i in dataPathLs])
    dataPathLs = dataPathLs[(dateLs < 4000) | ((dateLs > 300000) & (dateLs < 310000))]
    OrderLog = []
    ll = []
    
    for i in dataPathLs[:1]:
        try:
            df = pd.read_csv(i, encoding='GBK')
        except:
            print("empty data")
            print(i)
            ll.append(int(os.path.basename(i).split('.')[0]))
            continue
        df["SecurityID"] = int(os.path.basename(i).split('.')[0])
        OrderLog += [df]
    OrderLog = pd.concat(OrderLog).reset_index(drop=True)
    OrderLog = OrderLog[OrderLog["ChannelNo"] != 4001]
    
    OrderLog = OrderLog.rename(columns={"OrdType": "OrderType"})
    OrderLog["date"] = OrderLog["TransactTime"].iloc[0]//1000000000
    OrderLog["OrderType"] = np.where(OrderLog["OrderType"] == 'U', 3, OrderLog["OrderType"])
    OrderLog["skey"] = OrderLog["SecurityID"] + 2000000
    OrderLog["clockAtArrival"] = OrderLog["TransactTime"].astype(str).apply(lambda x: np.int64(datetime.datetime.strptime(x, '%Y%m%d%H%M%S%f').timestamp()*1e6))
    OrderLog['datetime'] = OrderLog["clockAtArrival"].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
    OrderLog["time"] = (OrderLog['TransactTime'] - int(OrderLog['TransactTime'].iloc[0]//1000000000*1000000000)).astype(np.int64)*1000
    
    for col in ["skey", "date", "ApplSeqNum", "OrderQty", "Side", "OrderType"]:
        OrderLog[col] = OrderLog[col].astype('int32')
    for cols in ["Price"]:
        OrderLog[cols] = OrderLog[cols].apply(lambda x: round(x, 2)).astype(np.float32)
    
    assert(OrderLog[((OrderLog["Side"] != 1) & (OrderLog["Side"] != 2)) | (OrderLog["OrderType"].isnull())].shape[0] == 0)
    
    OrderLog = OrderLog.rename(columns={"Side":"order_side", "OrderType":"order_type", "Price":"order_price", "OrderQty":"order_qty"})
    OrderLog = OrderLog[["skey", "date", "time", "clockAtArrival", "datetime", "ApplSeqNum", "order_side", "order_type", "order_price",
                                                 "order_qty"]]
    display(OrderLog.dtypes)
    display(OrderLog["order_side"].unique())
    display(OrderLog["order_type"].unique())
    print(datetime.datetime.now() - startTm)
#     OrderLog.to_pickle(r"\\192.168.10.30\Kevin_zhenyu\temp\20200103_order.pkl")

    
year = "2020"
startDate = '20200103'
endDate = '20200103'
readPath = 'L:\\DB\\' + year + '\\SZ\\***'
dataPathLs = np.array(glob.glob(readPath))
dateLs = np.array([os.path.basename(i) for i in dataPathLs])
dataPathLs = dataPathLs[(dateLs >= startDate) & (dateLs <= endDate)]

for data in dataPathLs:
    
    startTm = datetime.datetime.now()
    
    readPath = data + '\\tick\\***'
    dataPathLs = np.array(glob.glob(readPath))
    dateLs = np.array([int(os.path.basename(i).split('.')[0]) for i in dataPathLs])
    dataPathLs = dataPathLs[(dateLs < 4000) | ((dateLs > 300000) & (dateLs < 310000))]
    TradeLog = []
    ll = []
    
    for i in dataPathLs[:1]:
        try:
            df = pd.read_csv(i)
        except:
            print("empty data")
            print(i)
            ll.append(int(os.path.basename(i).split('.')[0]))
            continue
        df["SecurityID"] = int(os.path.basename(i).split('.')[0])
        TradeLog += [df]
    TradeLog = pd.concat(TradeLog).reset_index(drop=True)
    TradeLog = TradeLog[TradeLog["ChannelNo"] != 4001]
    
    TradeLog["date"] = TradeLog["TransactTime"].iloc[0]//1000000000
    TradeLog = TradeLog.rename(columns={"Qty":"trade_qty", "Price":"trade_price", "ExecType":"trade_type"})
    TradeLog["trade_money"] = TradeLog["trade_price"] * TradeLog["trade_qty"]
    TradeLog["trade_flag"] = 0
    TradeLog["skey"] = TradeLog["SecurityID"] + 2000000
    TradeLog["clockAtArrival"] = TradeLog["TransactTime"].astype(str).apply(lambda x: np.int64(datetime.datetime.strptime(x, '%Y%m%d%H%M%S%f').timestamp()*1e6))
    TradeLog['datetime'] = TradeLog["clockAtArrival"].apply(lambda x: datetime.datetime.fromtimestamp(x/1e6))
    TradeLog["time"] = (TradeLog['TransactTime'] - int(TradeLog['TransactTime'].iloc[0]//1000000000*1000000000)).astype(np.int64)*1000
    TradeLog["trade_type"] = np.where(TradeLog["trade_type"] == 'F', 1, TradeLog["trade_type"])
    for col in ["skey", "date", "ApplSeqNum", "BidApplSeqNum", "OfferApplSeqNum", "trade_qty", "trade_type"]:
        TradeLog[col] = TradeLog[col].astype('int32')
    for cols in ["trade_price"]:
        TradeLog[cols] = TradeLog[cols].apply(lambda x: round(x, 2)).astype(np.float32)
        TradeLog[cols] = TradeLog[cols].astype(np.float32)
    for cols in ["trade_money"]:
        TradeLog[cols] = TradeLog[cols].apply(lambda x: round(x, 2)).astype(np.float64)
        TradeLog[cols] = TradeLog[cols].astype(np.float64)
        
    TradeLog = TradeLog[["skey", "date", "time", "clockAtArrival", "datetime", "ApplSeqNum", "trade_type", "trade_flag",
                                                 "trade_price", "trade_qty", "trade_money", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]]
    display(TradeLog.dtypes)
    display(TradeLog["trade_type"].unique())
    display(TradeLog["trade_flag"].unique())
    print(datetime.datetime.now() - startTm)
#     TradeLog.to_pickle(r"\\192.168.10.30\Kevin_zhenyu\temp\20200103_trade.pkl")
    
    


skey                       int32
date                       int32
time                       int64
clockAtArrival             int64
datetime          datetime64[ns]
ApplSeqNum                 int32
order_side                 int32
order_type                 int32
order_price              float32
order_qty                  int32
dtype: object

array([1, 2], dtype=int64)

array([2, 1, 3], dtype=int64)

0:00:01.109033


skey                        int32
date                        int32
time                        int64
clockAtArrival              int64
datetime           datetime64[ns]
ApplSeqNum                  int32
trade_type                  int32
trade_flag                  int64
trade_price               float32
trade_qty                   int32
trade_money               float64
BidApplSeqNum               int32
OfferApplSeqNum             int32
dtype: object

array([4, 1], dtype=int64)

array([0], dtype=int64)

0:00:00.076794


In [1]:
import pandas as pd
import random
import numpy as np
import glob
import os
import pickle
import datetime

startTm = datetime.datetime.now()

readPath = r'\\192.168.10.30\Kevin_zhenyu\day_stock_20200424\***'
dataPathLs = np.array(glob.glob(readPath))
dataPathLs = dataPathLs[[np.array([os.path.basename(i).split('.')[0][:2] == 'SH' for i in dataPathLs])]]
db = pd.DataFrame()
for p in dataPathLs:
    dayData = pd.read_csv(p, compression='gzip')
    db = pd.concat([db, dayData])

C:\Users\win\Anaconda3\lib\site-packages\ipykernel_launcher.py:13: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  del sys.path[0]


In [3]:
pd.set_option("max_columns", 200)
db[db["date"] == '2020-01-03'].groupby("ID")

,ID,date,d_open,d_high,d_low,d_close,d_yclose,d_cumprodCAA,d_dayReturn,d_5dayReturn,d_ICDayReturn,d_CSIDayReturn,d_volume,d_amount,TORate,allZT,hasZT,isZT,allDT,hasDT,isDT,tmrHalted,haltedDays,marketShares,totalShares
0,SH603693,2018-07-03,10.80,12.96,10.80,12.96,9.00,1.00000,0.440000,0.440000,0.011355,0.011466,69218.0,892745.0,0.000587,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,118000000.0,618000000.0
1,SH603693,2018-07-04,14.26,14.26,14.26,14.26,12.96,1.00000,0.100309,0.584444,-0.014583,-0.018017,44453.0,633900.0,0.000377,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,118000000.0,618000000.0
2,SH603693,2018-07-05,15.69,15.69,15.69,15.69,14.26,1.00000,0.100281,0.743333,-0.027059,-0.026907,107022.0,1679175.0,0.000907,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,118000000.0,618000000.0
3,SH603693,2018-07-06,17.26,17.26,17.26,17.26,15.69,1.00000,0.100064,0.917778,0.002783,0.004095,323661.0,5586389.0,0.002743,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,118000000.0,618000000.0
4,SH603693,2018-07-09,18.99,18.99,18.99,18.99,17.26,1.00000,0.100232,1.110000,0.022199,0.021019,673553.0,12790771.0,0.005708,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,118000000.0,618000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656,SH603535,2020-04-20,16.00,16.19,15.89,16.03,16.03,0.98689,0.000000,0.038885,0.011908,0.016348,2305400.0,36922846.0,0.035406,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,65112500.0,150400000.0
657,SH603535,2020-04-21,16.05,16.13,15.82,16.00,16.03,0.98689,-0.001871,-0.057160,-0.007043,-0.005237,1964188.0,31336227.0,0.030166,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,65112500.0,150400000.0
658,SH603535,2020-04-22,15.90,16.10,15.80,16.10,16.00,0.98689,0.006250,-0.047337,0.008186,0.009276,1480600.0,23637912.0,0.022739,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,65112500.0,150400000.0
659,SH603535,2020-04-23,16.22,16.76,16.20,16.47,16.10,0.98689,0.022981,-0.062607,-0.005537,-0.006364,2533347.0,41721258.0,0.038907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,65112500.0,150400000.0


In [4]:
t2 = pd.read_pickle(r"\\192.168.10.30\Kevin_zhenyu\temp\KR data format\20200103_trade.pkl")
t2["cum_volume"] = t2[t2["trade_type"] == 1].groupby("skey")["trade_quantity"].transform("sum")
t2["cum_amount"] = t2[t2["trade_type"] == 1].groupby("skey")["trade_money"].transform("sum")
t2 = t2.groupby("skey")["cum_volume", "cum_amount"].first().reset_index()

In [6]:
t2

,skey,cum_volume,cum_amount
0,2000001,111619481.0,1.914495e+09
1,2000002,80553629.0,2.584310e+09
2,2000004,1338009.0,2.977932e+07
3,2000005,8179044.0,2.566005e+07
4,2000006,16565193.0,9.046555e+07
...,...,...,...
2178,2300807,214251.0,1.414914e+07
2179,2300808,4118807.0,1.024460e+08
2180,2300809,3551588.0,9.706851e+07
2181,2300810,4605795.0,3.524239e+08


In [ ]:
s2 = pd.read_pickle(r"\\192.168.10.30\Kevin_zhenyu\temp\KR data format\20200103_SZ.pkl")
s1 = pd.read_csv(r"\\192.168.10.30\Kevin_zhenyu\rawData\logs_20200103_zs_92_01_day_data\mdLog_SZ_20200103_0836.csv", nrows=100000)
display(s2[(s2["skey"] == 1) & (s2["time"] > 93000000000)].head(10))
display(s1[(s1["StockID"] == 1) & (s1["time"] > 93000000)].head(10))

In [ ]:
s2 = s2.groupby(["skey"])["cum_volume", "cum_amount"].max().reset_index()
t2 = pd.read_pickle(r"\\192.168.10.30\Kevin_zhenyu\temp\KR data format\20200103_trade.pkl")
t2["cum_volume"] = t2[t2["trade_type"] == 1].groupby("skey")["trade_quantity"].transform("sum")
t2["cum_amount"] = t2[t2["trade_type"] == 1].groupby("skey")["trade_money"].transform("sum")
t2 = t2.groupby("skey")["cum_volume", "cum_amount"].first().reset_index()
re = pd.merge(s2, t2, on="skey", how="outer")
display(re[re["cum_amount_x"].isnull()])
display(re[re["cum_amount_y"].isnull()])
display(re[re["cum_amount_x"] != re["cum_amount_y"]])
display(re[re["cum_volume_x"] != re["cum_volume_y"]])

In [11]:
TradeLog["OfferApplSeqNum"].max()

8005847

In [17]:
TradeLog["ExecType"].unique()

array(['4', 'F'], dtype=object)

In [14]:
    for col in ["skey", "date", "TransactTime", "ApplSeqNum", "BidApplSeqNum", "OfferApplSeqNum", "TradeQty"]:
        TradeLog[col] = TradeLog[col].astype('int32')
    for cols in ["TradePrice", "TradeMoney"]:
        TradeLog[cols] = TradeLog[cols].apply(lambda x: round(x, 2)).astype(np.float32)

In [5]:
TradeLog[TradeLog["ExecType"] != '4'].head(5)

,ApplSeqNum,BidApplSeqNum,SendingTime,TradePrice,ChannelNo,TradeQty,OfferApplSeqNum,Amt,ExecType,TransactTime,SecurityID,date,TradeMoney,TradeBSFlag,skey,clockAtArrival,datetime
19,116131,182,20190102092500000,9.39,2011,106,8257,0.0,F,92500000,1,20190102,995.34,N,2000001,1546392300000000,2019-01-02 09:25:00
20,116132,182,20190102092500000,9.39,2011,3594,61582,0.0,F,92500000,1,20190102,33747.66,N,2000001,1546392300000000,2019-01-02 09:25:00
21,116133,7907,20190102092500000,9.39,2011,7100,61582,0.0,F,92500000,1,20190102,66669.00,N,2000001,1546392300000000,2019-01-02 09:25:00
22,116134,8106,20190102092500000,9.39,2011,18,61582,0.0,F,92500000,1,20190102,169.02,N,2000001,1546392300000000,2019-01-02 09:25:00
23,116135,8106,20190102092500000,9.39,2011,2982,76225,0.0,F,92500000,1,20190102,28000.98,N,2000001,1546392300000000,2019-01-02 09:25:00


In [16]:
TradeLog[["skey", "date", "TransactTime", "clockAtArrival", "datetime", "ApplSeqNum", "ExecType", "TradeBSFlag",
                                                 "TradePrice", "TradeQty", "TradeMoney", "BidApplSeqNum",
                                                 "OfferApplSeqNum"]].dtypes

skey                        int32
date                        int32
TransactTime                int32
clockAtArrival              int64
datetime           datetime64[ns]
ApplSeqNum                  int32
ExecType                   object
TradeBSFlag                object
TradePrice                float32
TradeQty                    int32
TradeMoney                float32
BidApplSeqNum               int32
OfferApplSeqNum             int32
dtype: object

In [12]:
TradeLog["TradePrice"].astype(str).apply(lambda x: len(str(x.split('.')[1]))).unique()

array([1, 2], dtype=int64)

In [9]:
OrderLog[OrderLog["ExpirationDays"] != 0]

,OrderQty,OrderType,TransactTime,ExpirationDays,Side,ApplSeqNum,Contactor,SendingTime,Price,ChannelNo,ExpirationType,ContactInfo,ConfirmID,SecurityID,date,skey,clockAtArrival,datetime
716896,1000000,NaN,143002760,28,F,13,NaN,20190102143002000,1.8,4001,1,NaN,NaN,338,20190102,2000338,1546410602760000,2019-01-02 14:30:02.760
2271178,300000,NaN,132037000,28,F,5,NaN,20190102132036000,1.8,4001,1,NaN,NaN,728,20190102,2000728,1546406437000000,2019-01-02 13:20:37.000
2272447,1000000,NaN,140520530,3,F,7,NaN,20190102140520000,1.5,4001,1,NaN,NaN,728,20190102,2000728,1546409120530000,2019-01-02 14:05:20.530
2272450,1000000,NaN,140526730,7,F,8,NaN,20190102140526000,1.6,4001,1,NaN,NaN,728,20190102,2000728,1546409126730000,2019-01-02 14:05:26.730
2272453,1000000,NaN,140532950,14,F,9,NaN,20190102140532000,1.7,4001,1,NaN,NaN,728,20190102,2000728,1546409132950000,2019-01-02 14:05:32.950
2272457,1000000,NaN,140539180,28,F,10,NaN,20190102140539000,1.8,4001,1,NaN,NaN,728,20190102,2000728,1546409139180000,2019-01-02 14:05:39.180
2603514,400000,NaN,101142320,14,F,4,NaN,20190102101142000,1.7,4001,1,NaN,NaN,776,20190102,2000776,1546395102320000,2019-01-02 10:11:42.320
3711015,110000,NaN,141843220,7,F,11,NaN,20190102141843000,1.6,4001,1,NaN,NaN,983,20190102,2000983,1546409923220000,2019-01-02 14:18:43.220
4444365,12000,NaN,135129130,14,F,6,NaN,20190102135129000,1.7,4001,1,NaN,NaN,2051,20190102,2002051,1546408289130000,2019-01-02 13:51:29.130
5496396,20000,NaN,142840790,14,F,12,NaN,20190102142840000,1.7,4001,1,NaN,NaN,2179,20190102,2002179,1546410520790000,2019-01-02 14:28:40.790


In [29]:
OrderLog[["skey", "date", "TransactTime", "clockAtArrival", "datetime", "ApplSeqNum", "Side", "OrderType", "Price",
                                                 "OrderQty"]].dtypes

skey                       int32
date                       int32
TransactTime               int32
clockAtArrival             int64
datetime          datetime64[ns]
ApplSeqNum                 int32
Side                      object
OrderType                 object
Price                    float32
OrderQty                   int32
dtype: object

In [17]:
data

'G:\\KR\\2019\\SZ\\20190102'

In [15]:
year = "2019"
startDate = '20190102'
endDate = '20190102'
readPath = 'G:\\KR\\' + year + '\\SZ\\***'
dataPathLs = np.array(glob.glob(readPath))
os.path.basename(dataPathLs[0])

'20190102'

In [13]:
dateLs

array([], dtype=float64)

In [11]:
year = "2019"
startDate = '20190102'
endDate = '20190102'
readPath = 'G:\\KR\\' + year + '\\SZ\\***'
dataPathLs = np.array(glob.glob(readPath))
np.array([os.path.basename(i).split('.')[0] for i in dataPathLs])

array(['20190102', '20190103', '20190104', '20190107', '20190108',
       '20190109', '20190110', '20190111', '20190114', '20190115',
       '20190121', '20190122', '20190123', '20190124', '20190125',
       '20190128', '20190129', '20190130', '20190131', '20190201',
       '20190211', '20190212', '20190213', '20190214', '20190215',
       '20190218', '20190219', '20190220', '20190221', '20190222',
       '20190225', '20190226', '20190227', '20190228', '20190301',
       '20190304', '20190305', '20190306', '20190307', '20190308',
       '20190311', '20190312', '20190313', '20190314', '20190315',
       '20190318', '20190319', '20190320', '20190321', '20190322',
       '20190325', '20190326', '20190327', '20190328', '20190329',
       '20190401', '20190402', '20190403', '20190404', '20190408',
       '20190409', '20190410', '20190411', '20190412', '20190415',
       '20190416', '20190417', '20190418', '20190419', '20190422',
       '20190423', '20190424', '20190425', '20190426', '201904

In [9]:
dateLs

array([], dtype=float64)

In [7]:
year = "2019"
startDate = '20190102'
endDate = '20190102'
readPath = 'G:\\KR\\' + year + '\\SZ\\***'
dataPathLs = np.array(glob.glob(readPath))
os.path.basename(dataPathLs[0]).split('.')[0]

'20190102'

In [3]:
dateLs

array([], dtype=float64)

In [2]:
dataPathLs

array([], dtype=float64)